In [1]:
import random
from pprint import pprint

import numpy as np

from utils import read_data
from constants import FILENAME_EXAMPLE_1

In [2]:
input_data = read_data(FILENAME_EXAMPLE_1)

days    = input_data['length_of_schedule']
ne      = input_data['number_of_employees']
ns      = input_data['number_of_shifts']
demand  = input_data['temporal_requirements_matrix']
sn      = input_data['shift_name']
ss      = input_data['start_shift']
ls      = input_data['length_shift']
min_ls  = input_data['min_length_of_blocks']
max_ls  = input_data['max_length_of_blocks']
min_do  = input_data['min_days_off']
max_do  = input_data['max_days_off']
min_lw  = input_data['min_length_work_blocks']
max_lw  = input_data['max_length_work_blocks']
nf2     = input_data['nr_sequences_of_length_2']
nf3     = input_data['nr_sequences_of_length_3']
f2      = input_data['not_allowed_shift_sequences_2']
f3      = input_data['not_allowed_shift_sequences_3']

In [3]:
input_data

{'length_of_schedule': 7,
 'number_of_employees': 9,
 'number_of_shifts': 3,
 'temporal_requirements_matrix': [[2, 2, 2, 2, 2, 2, 2],
  [2, 2, 2, 3, 3, 3, 2],
  [2, 2, 2, 2, 2, 2, 2]],
 'shift_name': ['D', 'A', 'N'],
 'start_shift': [[3, 6, 0], [8, 4, 0], [1, 3, 2, 0]],
 'length_shift': [[4, 8, 0], [4, 8, 0], [4, 8, 0]],
 'min_length_of_blocks': [2, 2, 2],
 'max_length_of_blocks': [7, 6, 4],
 'min_days_off': 2,
 'max_days_off': 4,
 'min_length_work_blocks': 4,
 'max_length_work_blocks': 7,
 'nr_sequences_of_length_2': 3,
 'nr_sequences_of_length_3': 0,
 'not_allowed_shift_sequences_2': [['N', 'D'], ['N', 'A'], ['A', 'D']],
 'not_allowed_shift_sequences_3': []}

In [4]:
shifts = ns + 1
day, afternoon, night, dayoff = 1, 2, 3, 4
code = sn + ['-']

In [5]:
code

['D', 'A', 'N', '-']

In [6]:
def calculate_result(input_data):
    return [[random.choice(code) for d in range(days)] for e in range(ne)]

In [7]:
code

['D', 'A', 'N', '-']

In [8]:
calculate_result(input_data)

[['A', 'N', 'N', 'A', 'D', 'N', 'A'],
 ['N', 'N', 'A', 'A', '-', '-', 'A'],
 ['-', '-', 'A', 'N', '-', 'D', 'N'],
 ['A', '-', 'D', 'D', 'D', 'N', 'A'],
 ['-', 'N', 'A', 'D', 'N', '-', 'N'],
 ['D', 'N', 'N', 'N', '-', 'D', 'A'],
 ['A', 'N', 'A', 'N', 'A', 'D', '-'],
 ['-', 'D', '-', 'N', 'A', 'D', '-'],
 ['N', 'A', '-', 'N', '-', 'A', '-']]

In [9]:
code

['D', 'A', 'N', '-']

In [10]:
def demand_constraint(result):
    #sum(e in EMPLOYEE)(x[e,d] == day) == demand[1, d] /\
    #sum(e in EMPLOYEE)(x[e,d] == afternoon) == demand[2, d] /\
    #if length(minShift) == 3 then sum(e in EMPLOYEE)(x[e,d] == night) == demand[3, d] endif
    
    for d in range(days):
        sum_day, sum_afternoon, sum_night = 0, 0, 0
        for e in range(ne):
            if result[e][d] == 'D': sum_day += 1
            elif result[e][d] == 'A': sum_afternoon += 1
            elif result[e][d] == 'N': sum_night += 1
        
        if sum_day >= demand[0][d] and sum_afternoon >= demand[1][d] and sum_night >= demand[2][d]:
            pass
        else:
            return False
    return True

In [11]:
def update_demand_constraint(result):
    updated_result = result
    for d in range(days):
        sum_day, sum_afternoon, sum_night = 0, 0, 0
        for e in range(ne):
            if updated_result[e][d] == 'D': sum_day += 1
            elif updated_result[e][d] == 'A': sum_afternoon += 1
            elif updated_result[e][d] == 'N': sum_night += 1

        if sum_day >= demand[0][d]:
            pass
        else:
            updated_result[random.randint(0, ne-1)][d] = 'D'
        if sum_afternoon >= demand[1][d]:
            pass
        else:
            updated_result[random.randint(0, ne-1)][d] = 'A'

        if sum_night >= demand[2][d]:
            pass
        else:
            updated_result[random.randint(0, ne-1)][d] = 'N'
    return updated_result

In [12]:
#day off constraint
def day_off_constraint(result):
    amount=0
    for ii in range(len(result)):
        for i in range(7):
            if result[ii][i]==4:
                amount+=1
    if min_do<=amount<=max_do:
        return True
    else:
        return False

In [13]:
#working days in a row constraint
def length_work_blocks_constraint(result):
    amount=0
    for ii in range(len(result)):
        for i in range(6):
            if result[ii][i]!=4 and result[ii][i+1]!=4:
                amount+=1
            else:
                amount==0
    if min_lw<=amount<=max_lw:
        return True
    else:
        return False

In [14]:
#forbidden shifts constraint
def forbidden_constraint2(result):
    if f2!=[]:
        for ii in range(len(result)):
            for i in range(7):
                if result[ii][i]==3 and i<7:
                    result[ii][i+1]!=1 or result[i+1]!=2
                elif result[ii][i]==2 and i<7:
                    result[ii][i+1]!=1
        return True
    else:
        return False

In [15]:
def forbidden_constraint3(result):
    if f3!=[]:
        for ii in range(len(result)):
            for i in range(7):
                if result[ii][i]==3 and result[ii][i+1]==4 and i<6:
                    result[ii][i+2]!=3 or result[ii][i+2]!=1 or result[ii][i+2]!=2
                elif result[ii][i]==2 and result[ii][i+1]==4 and i<6:
                    result[ii][i+2]!=1
        return True
    else:
        return False

In [16]:
calculate_result(input_data)

[['A', 'A', '-', 'A', '-', '-', 'N'],
 ['-', '-', 'D', 'N', 'A', 'A', 'A'],
 ['-', 'N', 'D', 'A', 'N', '-', 'N'],
 ['-', '-', 'D', '-', 'A', 'N', '-'],
 ['D', 'D', 'D', 'N', 'D', '-', 'A'],
 ['N', 'N', 'A', '-', 'N', '-', 'D'],
 ['N', 'D', 'N', 'N', 'D', 'N', 'A'],
 ['A', 'N', '-', 'N', 'A', 'N', 'A'],
 ['A', '-', 'A', '-', '-', 'A', '-']]

In [17]:
fake_data = [
['D', 'D', 'D', 'D', 'D', 'D', 'D'],
['D', 'D', 'D', 'D', 'D', 'D', 'D'],
['A', 'A', 'A', 'A', 'A', 'A', 'A'],
['A', 'A', 'A', 'A', 'A', 'A', 'A'],
['A', 'A', 'A', 'A', 'A', 'A', 'A'],
['-', '-', '-', '-', '-', '-', '-'],
['-', '-', '-', '-', '-', '-', '-'],
['N', 'N', 'N', 'N', 'N', 'N', 'N'],
['N', 'N', 'N', 'N', 'N', 'N', 'N']]

In [18]:
pprint(demand)

[[2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 3, 3, 3, 2], [2, 2, 2, 2, 2, 2, 2]]


In [19]:
fake_data

[['D', 'D', 'D', 'D', 'D', 'D', 'D'],
 ['D', 'D', 'D', 'D', 'D', 'D', 'D'],
 ['A', 'A', 'A', 'A', 'A', 'A', 'A'],
 ['A', 'A', 'A', 'A', 'A', 'A', 'A'],
 ['A', 'A', 'A', 'A', 'A', 'A', 'A'],
 ['-', '-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-', '-'],
 ['N', 'N', 'N', 'N', 'N', 'N', 'N'],
 ['N', 'N', 'N', 'N', 'N', 'N', 'N']]

In [20]:
#result = calculate_result(input_data)
result = fake_data
pprint(result)
print(demand_constraint(result))
print(day_off_constraint(result))
print(length_work_blocks_constraint(result))
print(forbidden_constraint2(result))
print(forbidden_constraint3(result))

[['D', 'D', 'D', 'D', 'D', 'D', 'D'],
 ['D', 'D', 'D', 'D', 'D', 'D', 'D'],
 ['A', 'A', 'A', 'A', 'A', 'A', 'A'],
 ['A', 'A', 'A', 'A', 'A', 'A', 'A'],
 ['A', 'A', 'A', 'A', 'A', 'A', 'A'],
 ['-', '-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-', '-'],
 ['N', 'N', 'N', 'N', 'N', 'N', 'N'],
 ['N', 'N', 'N', 'N', 'N', 'N', 'N']]
True
False
False
True
False


In [21]:
result = calculate_result(input_data)

In [22]:
demand_constraint(result)

False

In [26]:
def solve_problem(input_data):
    result = calculate_result(input_data)
    while True:
        if demand_constraint(result):
            update_result = update_demand_constraint(result)
            break
        else:
            result = update_demand_constraint(result)
    return result

In [28]:
solution = solve_problem(input_data)
solution

[['-', 'N', 'D', '-', '-', 'D', 'D'],
 ['D', '-', 'N', 'D', 'D', 'N', 'N'],
 ['-', '-', 'D', 'N', 'D', 'N', 'A'],
 ['N', 'D', 'N', 'A', '-', 'D', 'N'],
 ['D', 'N', 'N', 'A', 'N', 'A', '-'],
 ['A', 'D', '-', 'N', 'A', '-', 'A'],
 ['N', 'A', 'N', 'A', 'A', '-', 'D'],
 ['N', 'A', 'A', '-', 'N', 'A', '-'],
 ['A', 'A', 'A', 'D', 'A', 'A', 'D']]

In [29]:
demand_constraint(solution)

True